In [65]:
import sys
print(sys.version)

3.10.6 (main, Aug 11 2022, 13:36:31) [Clang 13.1.6 (clang-1316.0.21.2.5)]


In [66]:
%load_ext autoreload
%autoreload 0

import sys
import os
import json

from draftfast import rules
from draftfast.optimize import run
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.settings import OptimizerSettings, CustomRule, PlayerPoolSettings
from draftfast.lineup_constraints import LineupConstraints
import pandas as pd
import numpy as np
from scipy import stats
from spread import get_metabet_spread, get_current_rankings, get_fantasy_def_points_against
from collections import defaultdict

# https://www.pro-football-reference.com/years/2021/fantasy.htm

SALARY_FILE = './data/9_11.csv'
WEEK = 1
MIN_SALARY = 4799
WEIGHTED = True

ACTIVE_FILE = './data/active.csv'
MAX_PLAYED = WEEK - 1
print('ready')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ready


In [67]:
# https://www.lineups.com/nfl-team-rankings
ranking_df = get_current_rankings(WEEK, 'MTM0ODI2MDkxMg==', True)

def get_abbr(x):
    try:
        if type(x) is not dict:
            x = eval(x)
        return x['team_abbr']
    except Exception as e:
        print(e,x)
        

print(ranking_df.shape)
ranking_df['team'] = ranking_df['nav'].apply(get_abbr)
# DEF_KEYS = ['passing_interceptions_rank'
# ranking_df['def_rank'] = ranking_df.apply(lambda row: np.avg([row[k] for k in DEF_KEYS]))
num_teams = ranking_df.shape[0]


return cached data week 1
(32, 52)


In [68]:
rankings = {x['team']: x for x in ranking_df.to_dict('records')}
# rankings
teams = set(rankings.keys())
print(teams, len(teams))
rankings['LV'] = rankings['OAK']
rankings['JAC'] = rankings['JAX']
del rankings['OAK']
rankings['PHI']

{'BUF', 'HOU', 'MIN', 'PHI', 'ATL', 'OAK', 'DEN', 'TEN', 'SF', 'TB', 'IND', 'LAC', 'CIN', 'NE', 'JAX', 'CLE', 'GB', 'BAL', 'DAL', 'SEA', 'MIA', 'ARI', 'CHI', 'PIT', 'KC', 'NYJ', 'NYG', 'LAR', 'DET', 'CAR', 'NO', 'WAS'} 32


{'Unnamed: 0': 1,
 'offensive_rating_rank': 18,
 'points_def': 385,
 'red_zone_attempts_rank': 23,
 'passing_interceptions': 12,
 'red_zone_percentage': 66.7,
 'passing_interceptions_rank': 20,
 'red_zone_conversions': 40,
 'offensive_plays': 1076,
 'turnover': 16,
 'first_downs': 358,
 'fourth_down_percentage_rank': 11,
 'passing_yards': 3756,
 'rushing_attempts': 458,
 'team_fk__full_name': 'Philadelphia Eagles',
 'turnover_rank': 26,
 'offensive_yards_rank': 10,
 'red_zone_conversions_rank': 27,
 'third_down_percentage_rank': 23,
 'passing_attempts_rank': 16,
 'points': 30,
 'fourth_down_percentage': 48.3,
 'passing_touchdowns': 28,
 'sacks_rank': 31,
 'offensive_yards': 5590,
 'points_rank': 2,
 'sacks': 29,
 'offensive_plays_rank': 18,
 'overall_rating_rank': 12,
 'defensive_rating_rank': 18,
 'passing_completions': 409,
 'passing_completions_rank': 28,
 'passing_touchdowns_rank': 20,
 'rushing_attempts_rank': 18,
 'red_zone_percentage_rank': 29,
 'points_rank_def': 18,
 'special_

In [69]:

spread_df = get_metabet_spread(WEEK)
favor_map = {}
z_map = {}
z_scores = {}

if 'OverUnder' in spread_df.columns.values:
    points = list(spread_df['OverUnder'])
    zs = stats.zscore(points)
    for i, p in enumerate(points):
        z_scores[p] = zs[i]

# https://madduxsports.com/how-to-read-nfl-odds-lines.html"
for index, row in spread_df.iterrows():
    home, away = row['HomeTeam'], row['AwayTeam']
    favor_map[home] = row['PointSpread']
    favor_map[away] = -row['PointSpread']
    if 'OverUnder' in row:
        z_map[home] = z_scores[row['OverUnder']]
        z_map[away] = z_scores[row['OverUnder']]
    

    
if 'JAX' in favor_map:
    favor_map['JAC'] = favor_map['JAX']
    z_map['JAC'] = z_map['JAX']
    
if 'LVS' in favor_map:
    favor_map['LV'] = favor_map['LVS']
    z_map['LV'] = z_map['LVS']

print('favor_map teams', len(favor_map))
print(favor_map)
z_map

return cached data week 1
favor_map teams 33
{'LAR': 2.5, 'BUF': -2.5, 'ATL': 5.46, 'NO': -5.46, 'CHI': 6.65, 'SF': -6.65, 'CIN': -6.5, 'PIT': 6.5, 'DET': 3.96, 'PHI': -3.96, 'MIA': -2.65, 'NE': 2.65, 'NYJ': 6.63, 'BAL': -6.63, 'WAS': -3.36, 'JAC': 3.36, 'CAR': -2.35, 'CLE': 2.35, 'HOU': 8.0, 'IND': -8.0, 'TEN': -5.5, 'NYG': 5.5, 'MIN': 1.95, 'GB': -1.95, 'ARI': 3.5, 'KC': -3.5, 'LAC': -3.48, 'LVS': 3.48, 'DAL': 1.5, 'TB': -1.5, 'SEA': 6.07, 'DEN': -6.07, 'LV': 3.48}


{'LAR': 1.6863248839568918,
 'BUF': 1.6863248839568918,
 'ATL': -0.9206020490668418,
 'NO': -0.9206020490668418,
 'CHI': -1.1584965220423902,
 'SF': -1.1584965220423902,
 'CIN': -0.4200324288474551,
 'PIT': -0.4200324288474551,
 'DET': 0.5761506767376603,
 'PHI': 0.5761506767376603,
 'MIA': -0.17718182101824884,
 'NE': -0.17718182101824884,
 'NYJ': -0.41012015914014077,
 'BAL': -0.41012015914014077,
 'WAS': -0.6381023624083764,
 'JAC': -0.6381023624083764,
 'CAR': -1.1584965220423902,
 'CLE': -1.1584965220423902,
 'HOU': -0.45968150767671256,
 'IND': -0.45968150767671256,
 'TEN': -0.655448834396177,
 'NYG': -0.655448834396177,
 'MIN': 0.5464138676157154,
 'GB': 0.5464138676157154,
 'ARI': 1.8226185924324665,
 'KC': 1.8226185924324665,
 'LAC': 1.5525092429081453,
 'LVS': 1.5525092429081453,
 'DAL': 0.861128430822954,
 'TB': 0.861128430822954,
 'SEA': -1.046983487835102,
 'DEN': -1.046983487835102,
 'LV': 1.5525092429081453}

In [70]:
df = pd.read_csv(SALARY_FILE, na_values= '')
df = df.fillna(df.median())
print(df.describe())

df['Name'] = df['First Name'] + " " + df['Last Name']
df['Salary/FPPG'] = df['FPPG'] / df['Salary']

all_teams = favor_map.keys()
winning_teams = [x for x in all_teams if favor_map[x] < 0]
ordered_teams = sorted(all_teams, key=lambda x: favor_map[x])
print('ordered_teams', [(x, favor_map[x]) for x in ordered_teams])
# excluded_teams = set([x for x in all_teams if favor_map[x] > -MIN_FAVORED])
# top_teams = winning_teams
team_offset = 1
half_teams = int(len(ordered_teams)/2)
top_teams = ordered_teams#[team_offset:team_offset+int(half_teams)]
print(f"top_teams {top_teams}")
excluded_teams = set(all_teams) - set(top_teams)
print(f"excluded_teams {excluded_teams}")

questionable_players = list(df[(~df['Injury Indicator'].isna()) | (~df['Injury Details'].isna())]['Name'])
low_salary_players = list(df[((df['Salary'] < MIN_SALARY)) & (df['Position'] != 'D')]['Name'])

excluded_players = [*questionable_players, *low_salary_players]
excluded_teams = list(df[(df['Position'] == 'D') & df['Team'].isin(excluded_teams)]['Name'])

if excluded_teams:
    excluded_players.extend(excluded_teams)

# excluded_players = ["Rob Gronkowski"]
print(f"Excluding: {len(excluded_players)}")


custom_adds = []

# custom_adds = ['Travis Kelce', 'Clyde Edwards-Helaire', "Emmanuel Sanders", "D'Onta Foreman", "Van Jefferson","Rob Gronkowski", "Aaron Jones", "Giovani Bernard","George Kittle"]#, "Ja'Marr Chase"]
# custom_adds = ['AJ Dillon','Darrel Williams']#, 'Deebo Samuel']

if excluded_players:
    excluded_players.extend(custom_adds)
    
questionable_df = df[(df['Name'].isin(set(questionable_players)))]
    
excluded_df = df[(df['Name'].isin(set(excluded_players)))]
    
df = df[(~df['Name'].isin(set(excluded_players)))]# & (df['FPPG'] > 0) & (df['Played'] > 0)

             FPPG      Played        Salary  Tier  Unnamed: 14  Unnamed: 15
count  890.000000  890.000000    890.000000   0.0          0.0          0.0
mean     4.552185    8.608989   4935.280899   NaN          NaN          NaN
std      4.178111    5.069808    973.213907   NaN          NaN          NaN
min     -0.413333    1.000000   3000.000000   NaN          NaN          NaN
25%      2.137143    5.000000   4500.000000   NaN          NaN          NaN
50%      3.533333    8.000000   4500.000000   NaN          NaN          NaN
75%      5.405769   12.000000   5200.000000   NaN          NaN          NaN
max     23.782001   21.000000  10200.000000   NaN          NaN          NaN
ordered_teams [('IND', -8.0), ('SF', -6.65), ('BAL', -6.63), ('CIN', -6.5), ('DEN', -6.07), ('TEN', -5.5), ('NO', -5.46), ('PHI', -3.96), ('KC', -3.5), ('LAC', -3.48), ('WAS', -3.36), ('MIA', -2.65), ('BUF', -2.5), ('CAR', -2.35), ('GB', -1.95), ('TB', -1.5), ('DAL', 1.5), ('MIN', 1.95), ('CLE', 2.35), ('LAR', 2.5)

/var/folders/08/3gvkyj816j39hpywkn6fwnx80000gq/T/ipykernel_51223/3260510746.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.median())


In [71]:
REPLACE_MAP = {
    'LA': 'Los Angeles',
    '.':'',
}

def name_map(x):
    result = ' '.join(x.split(', ')[::-1])
    for k in REPLACE_MAP:
        result = result.replace(k, REPLACE_MAP[k])
    return result

In [72]:
print(winning_teams)

['BUF', 'NO', 'SF', 'CIN', 'PHI', 'MIA', 'BAL', 'WAS', 'CAR', 'IND', 'TEN', 'GB', 'KC', 'LAC', 'TB', 'DEN']


In [73]:
# http://rotoguru1.com/cgi-bin/fyday.pl?week=1&game=fd&scsv=1

start_week = WEEK-6# take last few games for momentum weighting.

file_names = [f"./history/week{week_number}.csv" for week_number in range(start_week, WEEK+1) if week_number != 18 and os.path.isfile(f"./history/week{week_number}.csv")]
print(file_names, len(file_names))

history_dfs = [pd.read_csv(f, delimiter=";") for f in file_names]
print(f"Using {len(history_dfs)} weeks of history")
historic_averages = {}
if history_dfs:
    historic_data=pd.concat(history_dfs)
    


    historic_data['Name'] = historic_data['Name'].apply(name_map)
    team_data = historic_data[historic_data['Pos'] == 'Def']

    historic_data[:1]
    historic_averages = historic_data.groupby("Name").mean()['FD points'].to_dict()
    historic_averages['Patrick Mahomes'] = historic_averages['Patrick Mahomes II']
    historic_averages['Darrell Henderson Jr'] = historic_averages['Darrell Henderson']
    # historic_averages

    team_averages = team_data.groupby("Team").mean()['FD points'].to_dict()
    team_averages['gb'] = team_averages.get('gnb')
    team_averages['kc'] = team_averages.get('kan')
    team_averages['ne'] = team_averages.get('nwe')
    team_averages['tb'] = team_averages.get('tam')
    team_averages['lv'] = team_averages.get('lvr')
    team_averages['no'] = team_averages.get('nor')
    team_averages['sf'] = team_averages.get('sfo')
    # print(team_averages)
    
    for k,v in team_averages.items():
        historic_averages[k] = team_averages[k]
    print(len(historic_averages))



[] 0
Using 0 weeks of history


In [74]:
excluded_bonus = defaultdict(lambda: 0)

for index, p in questionable_df.iterrows():
    pos = p['Position']
    if pos in ['TE', 'WR', 'RB', 'QB']:
        points = p['FPPG']
        if points > 7:
            if pos == 'QB':
                # subtract for QB
                amt = -min(points * .08, .8)
            else:
                amt = min(points * .05, .6)
            print('bonus', p['Team'], p['Name'], amt)
            
            excluded_bonus[p['Team']] += amt
        
print(excluded_bonus)

bonus SF Deebo Samuel 0.6
bonus CIN Joe Burrow -0.8
bonus CLE Deshaun Watson -0.8
bonus LV Darren Waller 0.4879166603088379
bonus CIN Tee Higgins 0.6
bonus BAL J.K. Dobbins 0.5308823529411764
bonus MIA Jaylen Waddle 0.6
bonus SF Eli Mitchell 0.6
bonus PIT Diontae Johnson 0.6
bonus CAR Sam Darnold -0.8
bonus NYJ Zach Wilson -0.8
bonus PHI Miles Sanders 0.4173076923076924
bonus NYG Tyrod Taylor -0.8
bonus JAC James Robinson 0.5657142639160156
bonus NO Michael Thomas 0.4427777608235677
bonus ARI DeAndre Hopkins 0.6
bonus ARI Zach Ertz 0.40916667514377175
bonus NE Jakobi Meyers 0.4464705972110524
bonus NYG Sterling Shepard 0.426428576878139
bonus BAL Gus Edwards 0.3658333248562283
bonus WAS Logan Thomas 0.44599998474121094
bonus CHI Byron Pringle 0.3631578947368421
bonus WAS J.D. McKissic 0.4836363705721768
bonus NE Kristian Wilkerson 0.6
bonus NO Tre'Quan Smith 0.3584999847412109
bonus ATL Calvin Ridley 0.555999984741211
defaultdict(<function <lambda> at 0x294f2fd00>, {'SF': 1.2, 'CIN': -

In [75]:
df.to_csv(ACTIVE_FILE)
print(f"wrote {ACTIVE_FILE}, {df.shape}")

wrote ./data/active.csv, (312, 19)


In [76]:
# https://www.fanduel.com/nfl-guide
# Use min/max + roster_size to accomodate flex position. FD roster_size = 9
def get_nfl_positions():
    return [
        ['QB', 1, 1],
        ['RB', 2 ,3],
        ['WR', 3, 4],
        ['TE', 1, 2],
        ['D', 1, 1]
    ]               

In [77]:
m_score = df.groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
    
m_score

{'D': 6.10631688572865,
 'QB': 7.131576994636367,
 'RB': 7.613981999988927,
 'TE': 7.653622854237035,
 'WR': 7.2240351045550515,
 'DEF': 6.10631688572865}

In [78]:
max_salary = np.percentile(df[df['Salary'] >= MIN_SALARY]['Salary'], 99)-1
max_salary = 9000
print(max_salary)

9000


In [79]:
df[df['Position'] == 'D'].shape

(26, 19)

In [80]:
ACTIVE_RULE_SET = rules.FD_NFL_RULE_SET
# Overrides (position limits, salary, roster size, positions, etc.
ACTIVE_RULE_SET.salary_max = 60000
ACTIVE_RULE_SET.salary_min = ACTIVE_RULE_SET.salary_max - 100
ACTIVE_RULE_SET.defensive_positions = ['D', 'DEF']
ACTIVE_RULE_SET.offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX']
ACTIVE_RULE_SET.position_limits = get_nfl_positions()
ACTIVE_RULE_SET.max_players_per_team = 9
ACTIVE_RULE_SET.roster_size = 9
print(ACTIVE_RULE_SET.__dict__)

ALL_POSITIONS = [*ACTIVE_RULE_SET.defensive_positions, *ACTIVE_RULE_SET.offensive_positions]

{'site': 'FAN_DUEL', 'league': 'NFL', 'roster_size': 9, 'position_limits': [['QB', 1, 1], ['RB', 2, 3], ['WR', 3, 4], ['TE', 1, 2], ['D', 1, 1]], 'general_position_limits': [], 'salary_min': 59900, 'salary_max': 60000, 'offensive_positions': ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX'], 'defensive_positions': ['D', 'DEF'], 'game_type': 'classic', 'max_players_per_team': 9, 'position_per_team_rules': None, 'min_teams': None}


In [81]:
# Any additional player or custom rule constraints.
#Player -  https://github.com/BenBrostoff/draftfast/blob/68625902ceea83e66ee9f13a44acd732f600f68f/draftfast/orm.py#L245

# no players min cost (unlikely to play), low score, or favored to lose except overwhelming proj.
# Use salary data from csv as optimization basis.
# set((k, v['defensive_rating_rank']) for k,v in rankings.items())
# rankings['BUF']

allowed_map = get_fantasy_def_points_against(WEEK)
allowed_map['Washington Commanders'] = allowed_map['Washington Football Team']
allowed_map

return cached data week 1


{'Jacksonville Jaguars': {'rank': 32, 'allowed': 9.71},
 'Atlanta Falcons': {'rank': 31, 'allowed': 9.59},
 'New York Giants': {'rank': 30, 'allowed': 9.59},
 'Carolina Panthers': {'rank': 29, 'allowed': 9.47},
 'Chicago Bears': {'rank': 28, 'allowed': 9.12},
 'Baltimore Ravens': {'rank': 27, 'allowed': 8.35},
 'New York Jets': {'rank': 26, 'allowed': 8.24},
 'Houston Texans': {'rank': 25, 'allowed': 7.65},
 'Washington Football Team': {'rank': 24, 'allowed': 7.53},
 'Denver Broncos': {'rank': 23, 'allowed': 7.53},
 'Miami Dolphins': {'rank': 22, 'allowed': 7.41},
 'Las Vegas Raiders': {'rank': 21, 'allowed': 6.94},
 'Detroit Lions': {'rank': 20, 'allowed': 6.65},
 'Cleveland Browns': {'rank': 19, 'allowed': 6.53},
 'New Orleans Saints': {'rank': 18, 'allowed': 6.47},
 'Tennessee Titans': {'rank': 17, 'allowed': 6.41},
 'Cincinnati Bengals': {'rank': 16, 'allowed': 6.24},
 'Pittsburgh Steelers': {'rank': 15, 'allowed': 6.06},
 'Los Angeles Rams': {'rank': 14, 'allowed': 5.71},
 'New En

In [82]:
players = salary_download.generate_players_from_csvs(salary_file_location=ACTIVE_FILE, game=rules.FAN_DUEL)

FAVOR_DIVISION = 8
AVERAGE_WEIGHT = .65
OVERUNDER_DIVISION = 3

defenses = []
qbs = []

historic_data_used = 0

for p in players:
    p.average_score = m_score[p.pos]
    name = p.name.replace('.', '')
    
    if WEIGHTED:
        base_score = p.proj
        average_score = p.proj
        
        history_key = name_map(p.name) if p.pos != 'D' else p.team.lower()
        history_value = historic_averages.get(history_key)
        if history_value:
            new_score = AVERAGE_WEIGHT*average_score + (1-AVERAGE_WEIGHT)*history_value
            base_score = new_score
            if 'Buffalo' in p.name:
                print(p.name, base_score, average_score, history_value)
            historic_data_used+=1
        else:
            new_score = None
            # print('no historic data', history_key)
            
        
        rank_bonus = 0
        
        # print('rank_bonus', rank_bonus)
        teams = p.matchup.split('@')
        
        
        is_home = p.team == teams[1]
        if is_home:
            rank_bonus += 0.2

        teams.remove(p.team) # remove current players team
        opponent = teams[0]
        
        point_bonus = z_map.get(p.team, 0)
        if point_bonus and p.pos != 'D':
            # max(0, point_bonus/4)
            overunder_bonus = -point_bonus/OVERUNDER_DIVISION if p.pos == 'D' else point_bonus/OVERUNDER_DIVISION # lower points better for defensive scoring.
            rank_bonus += overunder_bonus
            
        # current_rank = rankings[p.team]['overall_rating_rank']
        # opp_rank = rankings[opponent]['overall_rating_rank']
        current_rank = rankings[p.team]['defensive_rating_rank']
        opp_rank = num_teams - rankings[opponent]['offensive_rating_rank']

        if p.pos == 'D':
            opp_bonus = excluded_bonus[opponent]/2
            rank_bonus += opp_bonus
            opp_def_avg = allowed_map[rankings[opponent]['team_fk__full_name']]['allowed']
            base_score = AVERAGE_WEIGHT*base_score + (1-AVERAGE_WEIGHT)*opp_def_avg
        else:
            if p.pos == 'QB':
                # negative (downside for QB injuries)
                injury_bonus = -excluded_bonus[p.team]
            else:
                injury_bonus = excluded_bonus[p.team] 
                
            rank_bonus += injury_bonus
            rank_bonus += -favor_map.get(p.team, 0)/FAVOR_DIVISION
            
        # overall_diff = opp_rank - current_rank # larger the better (should be between 1-32)
        # rank_bonus += overall_diff/num_teams*1.5
        p.proj = base_score + rank_bonus
        
        if p.pos == 'D':
            defenses.append((p.team, average_score, rank_bonus, opp_def_avg, p.proj, p.cost, p.proj / p.cost))
        elif p.pos == 'QB':
            qbs.append((history_key, average_score, base_score, p.proj, p.cost, p.proj / p.cost))
        
        if 'Chase' in name or 'Uzomah' in name:# or p.pos == 'TE':
            print(history_key, history_value, base_score, overunder_bonus, p.proj, p.cost, p.proj / p.cost)
        

print(f"players {len(players)}")
print(f"historic data used {historic_data_used} of {len(players)}") 
# print(players)

for x in sorted(defenses, key=lambda x: x[-1], reverse=True):
    print(x)
# print(excluded_bonus)
print("\n---\n")
for x in sorted(qbs, key=lambda x: x[-1], reverse=True):
    print(x)

Ja'Marr Chase None 15.361905052548362 -0.14001080961581838 16.034394242932542 8200.0 0.0019554139320649444
Chase Daniel None -0.2000000029802322 0.5175030809693818 0.9525030779891497 6100.0 0.00015614804557199176
Chase Garbers None 3.533333333333333 0.5175030809693818 3.127919753993877 6000.0 0.0005213199589989795
Chase Edmonds None 9.584615267240084 -0.05906060700608295 10.656804660234 5800.0 0.0018373801138334483
Chase Claypool None 8.818750381469727 -0.14001080961581838 8.466239571853908 5800.0 0.0014596964779058463
CJ Uzomah None 6.594999694824219 -0.13670671971338025 5.029542975110839 4800.0 0.0010478214531480915
players 312
historic data used 0 of 312
('MIA', 9.0, 0.7232352986055262, 5.71, 8.571735298605526, 3800.0, 0.002255719815422507)
('NE', 8.777777777777779, 0.3, 7.41, 8.599055555555557, 3900.0, 0.0022048860398860403)
('WAS', 5.705882352941177, 0.48285713195800783, 9.71, 7.590180661369773, 3700.0, 0.002051400178748587)
('MIN', 7.352941176470588, 0.2, 3.53, 6.2149117647058825

In [87]:
players

[[RB] Jonathan Taylor      IND IND@HOU ($10,200, 21.617361425098878 (14.00)), ,
 [RB] Derrick Henry        TEN NYG@TEN ($9,700, 22.5023503885346 (14.89)), ,
 [RB] Christian McCaffrey  CAR CLE@CAR ($9,500, 14.879013064081107 (7.27)), ,
 [RB] Austin Ekeler        LAC LV@LAC ($9,400, 20.45250231802993 (12.84)), ,
 [QB] Patrick Mahomes      KC KC@ARI ($8,700, 24.827040263232696 (17.70)), ,
 [WR] Davante Adams        LV LV@LAC ($8,500, 17.99983078792344 (10.78)), ,
 [RB] Alvin Kamara         NO NO@ATL ($8,500, 17.423064007279077 (9.81)), ,
 [QB] Lamar Jackson        BAL BAL@NYJ ($8,500, 20.876993913117467 (13.75)), ,
 [RB] Dalvin Cook          MIN GB@MIN ($8,400, 14.699926652160968 (7.09)), ,
 [QB] Justin Herbert       LAC LV@LAC ($8,400, 24.43250365541791 (17.30)), ,
 [RB] Joe Mixon            CIN PIT@CIN ($8,300, 16.824489617630274 (9.21)), ,
 [RB] Najee Harris         PIT PIT@CIN ($8,200, 14.397489190384182 (6.78)), ,
 [WR] Ja'Marr Chase        CIN PIT@CIN ($8,200, 16.034394242932542 (8.

In [90]:
# list(filter(lambda p: 'Burrow' in p.name, players))
player_settings = PlayerPoolSettings()
LOCKED = []
BLOCKED_TEAMS = []

# overrides:
# LOCKED = ['Josh Allen', 'Tee Higgins', "Ja'Marr Chase"]#, 'Rob Gronkowski']#, 'C.J. Uzomah']#'Davante Adams']
# LOCKED = ['Eli Mitchell']#, 'Allen Lazard', "Ja'Marr Chase", "Tee Higgins"]
# LOCKED = ['Buffalo Bills','Josh Allen']
# LOCKED = ['Tee Higgins', 'Eli Mitchell', 'Buffalo Bills', 'Tyler Higbee', "Ja'Marr Chase"]
# LOCKED = ['Tee Higgins', 'Eli Mitchell', 'Buffalo Bills', "Ja'Marr Chase", "Josh Allen"]#, 'Tee Higgins']
# BLOCKED_TEAMS = []

constraints = LineupConstraints(locked=LOCKED)

MIN_PROJ = 5
min_favored = 0
MIN_LIMIT = -4

roster = None
best_roster = None
best_score = 0


get_score = lambda roster: sum([p.proj for p in roster.players])

while min_favored >= MIN_LIMIT:
    def block_function(p):
        store = p.kv_store
        played = int(float(store.get('Played', -1)))
        
        # should_skip = ((p.team in BLOCKED_TEAMS or p.proj < MIN_PROJ) and p.pos != 'D') or (p.cost > max_salary and p.pos != 'QB') or played < MAX_PLAYED/3 or (p.pos == 'QB' and p.proj < 12)# or (favor_map.get(p.team, 0) > -min_favored)
        should_skip = False
        #print(p.name, played, MAX_PLAYED)
        return should_skip

    opt_settings = OptimizerSettings(
        custom_rules=[
            CustomRule(
                group_a=lambda p: p,
                group_b=block_function, 
                comparison=lambda sum, a, b: sum(b) == 0
            ),
        ],
        min_teams=0#len(defenses)/2+1
    )

    roster = run(
        rule_set=ACTIVE_RULE_SET,
        player_pool=players,
        verbose=True,
        optimizer_settings=opt_settings,
        constraints=constraints,
        player_settings=player_settings
    )
    
    if roster:
        # total_salary = sum([p.cost for p in roster.players])
        print(roster)
        print('min_favored', min_favored)# negative means players' teams can lose
        print(f"spread weighted: {WEIGHTED}\n\n")
        current_score = get_score(roster)
        if not best_score or current_score > best_score:
            best_score = current_score
            best_roster = roster

    min_favored -= 1
    
    if min_favored >= MIN_LIMIT:
        roster = None
        
if not roster:
    print("No solution")
elif not favor_map:
    print("Warning: No favor map used")

Optimal roster for: NFL
+----------+----------------------+------+---------+--------+--------------------+----------+--------+
| Position | Player               | Team | Matchup | Salary |         Projection | vs. Avg. | Locked |
+----------+----------------------+------+---------+--------+--------------------+----------+--------+
| QB       | Justin Herbert       | LAC  | LV@LAC  |  8,400 |  24.43250365541791 |    17.30 |        |
| RB       | Derrick Henry        | TEN  | NYG@TEN |  9,700 |   22.5023503885346 |    14.89 |        |
| RB       | James Conner         | ARI  | KC@ARI  |  7,700 | 16.885456587424322 |     9.27 |        |
| WR       | Mike Williams        | LAC  | LV@LAC  |  6,600 |  14.19000346243911 |     6.97 |        |
| WR       | Adam Thielen         | MIN  | GB@MIN  |  6,100 | 12.930695882930198 |     5.71 |        |
| WR       | Hunter Renfrow       | LV   | LV@LAC  |  5,900 | 12.648197179971797 |     5.42 |        |
| WR       | Robert Woods         | TEN  | NYG@TE

In [84]:
# def find_id(first_name, last_name, pos):
#     matches = df.loc[(df['First Name'] == first_name) & (df['Last Name'] == last_name) & (df['Position'] == pos)]
#     if matches:||
#         return matches.iloc[0]['Id']
#     return None
#print(f"{"Player":20}{"Advantage":10}")
if favor_map:
    roster = best_roster
    sorted_players = sorted(roster.players, key=lambda x: favor_map[x.team])
    net_score = 0
    for p in sorted_players:
        advantage = favor_map[p.team]
        name = p.name.replace('.', '')
        print(f"{name:24}{advantage:>10}{historic_averages.get(name, ''):>20} {p.proj} {p.kv_store['Played']} {excluded_bonus[p.team]}")#, p.kv_store['Played'], MAX_PLAYED)
        net_score += advantage
    print(f"---\nTotal adv: {net_score}\n")
    roster.players[-1].__dict__



AttributeError: 'NoneType' object has no attribute 'players'

In [ ]:
for r in roster.players:
    print(r.team, r.name, r.proj, r.average_score, r.cost,  r.kv_store['Injury Indicator'], r.kv_store['Injury Details'], r.kv_store['Played'])
    # if 'Zay' in r.name:
    #     print(r.__dict__)

In [ ]:
# Create upload CSV by reordering columns to match template order.

headers = []
players = []
ORDERED_COLS = ['QB','RB','RB','WR','WR','WR','TE','FLEX','DEF'] # template order.

def get_match_names(col):
    if col == 'DEF':
        return ['D']
    elif col == 'FLEX' :
        return ['RB', 'WR']
    return [col]

roster_copy = roster.players.copy()
for c in ORDERED_COLS:
    headers.append(c)
    match_names = get_match_names(c)
    for r in roster_copy:
        if r.pos in match_names:
            p = f"{r.kv_store['Id']}:{r.name}"
            players.append(p)
            roster_copy.remove(r)
            break

with open('upload.csv', 'w') as f:
    f.write(','.join(headers))
    f.write('\n')
    f.write(','.join(players))

print('done')


In [ ]:
output = pd.read_csv('upload.csv')
output

In [ ]:
m_score = df[(df['FPPG'] > 0) & (df['Played'] > 0)].groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
m_score['FLEX'] = (m_score['WR']+m_score['RB'])/2
m_score

In [ ]:
expected_score = sum([m_score.get(h) for h in headers])
expected_score

In [ ]:
# Players that have non-null injury status.
df[(~df['Injury Indicator'].isna()) | ~df['Injury Details'].isna()]

In [ ]:
df[df['First Name'] == 'Zay']

In [ ]:
# Potentially unaccounted positions
df[~df['Position'].isin(ALL_POSITIONS)]

In [ ]:
df[df['Position'] == 'D'][['FPPG', 'Nickname', 'Salary', 'Salary/FPPG']].sort_values('Salary/FPPG', ascending=False)

In [ ]:
df[df['Team'] == 'JAC']

In [ ]:
spread_df.iloc[1][['HomeTeam', 'PointSpread']]

In [ ]:
from pydfs_lineup_optimizer import Site, Sport, get_optimizer

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)
optimizer.load_players_from_csv(ACTIVE_FILE)
for lineup in optimizer.optimize(10):
    print(lineup)
